**Лабораторная работа № 1 «Классификация изображений с помощью сверточных нейронных сетей»**

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from keras import backend as K

Настраиваем параметрый для test и train данных

In [2]:
train_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/"
test_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/"
shape_of_image = [100, 100, 3]
number_of_class = 131

Создаем источники данных для тренировки и валидации сети

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255, shear_range=0.3, horizontal_flip=True, zoom_range=0.3)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=shape_of_image[:2],
                                                    batch_size=32,
                                                    color_mode='rgb',
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=shape_of_image[:2],
                                                  batch_size=32,
                                                  color_mode='rgb',
                                                  class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


Создаем CNN и натраиваем ее

In [6]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

vgg16_model = VGG16(include_top=False,  input_shape=(100, 100, 3))

model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(131, activation="relu"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])

Обучаем модель

In [7]:
batch_size = 32
number_of_batch = 1600 // batch_size
hist = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = number_of_batch,
    epochs = 100,
    validation_data = test_generator,
    validation_steps = 800 // batch_size
)

2022-10-21 09:49:32.082434: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
50/50 [==============================] - 27s 511ms/step - loss: 4.7294 - acc: 0.0237 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 4.4742 - val_acc: 0.0350 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/100
50/50 [==============================] - 25s 500ms/step - loss: 4.0678 - acc: 0.0806 - f1_m: 0.0048 - precision_m: 0.0800 - recall_m: 0.0025 - val_loss: 3.3688 - val_acc: 0.1388 - val_f1_m: 0.0168 - val_precision_m: 0.2400 - val_recall_m: 0.0088
Epoch 3/100
50/50 [==============================] - 25s 493ms/step - loss: 3.1901 - acc: 0.1650 - f1_m: 0.0609 - precision_m: 0.4687 - recall_m: 0.0331 - val_loss: 2.5848 - val_acc: 0.3713 - val_f1_m: 0.1421 - val_precision_m: 0.9100 - val_recall_m: 0.0787
Epoch 4/100
50/50 [==============================] - 24s 483ms/step - loss: 2.6140 - acc: 0.2775 - f1_m: 0.1374 - precision_m: 0.5257 - recall_m: 0.0812 - val_loss: 1.8213 - val_acc: 0.5063 - val_f1_m: 

Сохраняем веса

In [ ]:
model.save_weights("trialv4.h5")